# Agilepy science API tutorial - VELA Region analysis

The purpose of this notebook is to show how to use the Agilepy scientific API. 


## The complete API documentation can be found [at this link](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html).

## Importing the library

In [1]:
from agilepy.api.AGAnalysis import AGAnalysis
from tqdm.notebook import tqdm

In [2]:
# Interactive plot
#%matplotlib widget

## Creating a configuration file

In [3]:
confFilePath = "$HOME/agilepy_conf.yaml"

In [4]:
AGAnalysis.getConfiguration(
    evtfile="$AGILE/agilepy-test-data/test_dataset_6.0/EVT/EVT.index",
    logfile="$AGILE/agilepy-test-data/test_dataset_6.0/LOG/LOG.index",
    confFilePath = confFilePath,
    userName = "user-xxx",
    sourceName = "vela-xxx",
    tmin = 150000000,
    tmax = 160000000,
    timetype = "TT",
    glon = 263.55,
    glat = -2.78,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 2
)

## Initialization of the class

In [5]:
ag = AGAnalysis(confFilePath)

2021-12-23 10:56:49,869 [INFO    ] [AgilepyLogger] File and Console loggers are active. Log file: /home/antonio/agilepy_analysis/user-xxx_vela-xxx_20211223-105649/logs/analysis_log.log


In [50]:
!cat /home/antonio/agilepy_analysis/user-xxx_vela-xxx_20211223-105649/logs/analysis_log.log

2021-12-23 10:56:49,869 [INFO    ] [AgilepyLogger] File and Console loggers are active. Log file: /home/antonio/agilepy_analysis/user-xxx_vela-xxx_20211223-105649/logs/analysis_log.log
2021-12-23 10:56:50,275 [INFO    ] [SourcesLibrary] Loaded 9 sources. Total sources: 9
2021-12-23 10:56:50,526 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.80893e-07]
2021-12-23 10:56:50,527 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.54772e-07]
2021-12-23 10:56:50,528 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.64949e-07]
2021-12-23 10:56:50,529 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [3.14104e-07]
2021-12-23 10:56:50,529 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [2.64853e-07]
2021-12-23 10:56:50,529 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.91737e-07]
2021-12-23 10:

## API: configuration file 

Print all options of the configuration file

In [6]:
ag.printOptions()

{ 'ap': {'radius': 3, 'timeslot': 3600},
  'input': { 'datapath': '$AGILE/agilepy_data',
             'evtQfile': '$AGILE/agilepy_data/EVT.qfile',
             'evtfile': '/home/antonio/anaconda3/envs/agilepydev/agiletools/agilepy-test-data/test_dataset_6.0/EVT/EVT.index',
             'logQfile': '$AGILE/agilepy_data/LOG.qfile',
             'logfile': '/home/antonio/anaconda3/envs/agilepydev/agiletools/agilepy-test-data/test_dataset_6.0/LOG/LOG.index'},
  'maps': { 'binsize': 0.25,
            'energybins': [[100, 10000]],
            'expstep': 4.0,
            'fovbinnumber': 1,
            'mapsize': 40,
            'offaxisangle': 30,
            'proj': 'ARC',
            'projtype': 'WCS',
            'spectralindex': 2.1,
            'timestep': 160,
            'useEDPmatrixforEXP': False},
  'mle': { 'contourpoints': 40,
           'edpcorrection': 0.75,
           'expratio_maxthr': 15,
           'expratio_minthr': 0,
           'expratio_size': 10,
           'expratioeva

Print a single section of the yaml configuration file

In [7]:
ag.printOptions("maps")

{ 'binsize': 0.25,
  'energybins': [[100, 10000]],
  'expstep': 4.0,
  'fovbinnumber': 1,
  'mapsize': 40,
  'offaxisangle': 30,
  'proj': 'ARC',
  'projtype': 'WCS',
  'spectralindex': 2.1,
  'timestep': 160,
  'useEDPmatrixforEXP': False}


Print a single option

In [8]:
ag.getOption("energybins")

[[100, 10000]]

Set some options

In [9]:
ag.setOptions(energybins=[[100,300], [300, 1000]], mapsize=50, binsize=0.4)

In [10]:
ag.printOptions("maps")

{ 'binsize': 0.4,
  'energybins': [[100, 300], [300, 1000]],
  'expstep': 4.0,
  'fovbinnumber': 1,
  'mapsize': 50,
  'offaxisangle': 30,
  'proj': 'ARC',
  'projtype': 'WCS',
  'spectralindex': 2.1,
  'timestep': 160,
  'useEDPmatrixforEXP': False}


## API: interacting with the Sources Library

### Loading the AGILE's second catalogue of gamma-ray sources

In [11]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 25))

2021-12-23 10:56:50,275 [INFO    ] [SourcesLibrary] Loaded 9 sources. Total sources: 9


In [12]:
type(sources[0])

agilepy.core.source.Source.PointSource

In [13]:
for s in sources:
    print(s)


-----------------------------------------------------------
 Source name: 2AGLJ1045-5954 (PointSource)
  * Spectrum type: PowerLaw
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 1.8089e-07
	- index : 2.0414
	- Source position (l,b): (287.697, -0.788886)
	- Distance from map center (deg): 24.2159

-----------------------------------------------------------
 Source name: 2AGLJ1048-5836 (PointSource)
  * Spectrum type: PowerLaw
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 1.5477e-07
	- index : 2.00039
	- Source position (l,b): (287.422, 0.423015)
	- Distance from map center (deg): 24.0778

-----------------------------------------------------------
 Source name: 2AGLJ1045-5735 (PointSource)
  * Spectrum type: PowerLaw
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 1.6495e-07
	- index : 2.25973
	- Source position (l,b): (286.672, 1.23211)
	- Distance from map center (deg): 23.4607

-------------------

### Alternative solution: loading the sources from a custom AGILE .txt file
The sources that are already present in the Sources Library will not be loaded twice.

In [14]:
sources_hypotesis = """
969.539e-08 263.585 -2.84083 1.71345 0 2 2AGLJ0835-4514 0.0 2 3913.06 1.34774 0.5 5 20 10000 0 100
10e-08 273.0 -1.1 2.1 1 2 NEWOURCE 0.0 0 0 0 0.5 5 20 10000 0 100
"""
with open("./sources_hyp.txt", "w") as sf:
    sf.write(sources_hypotesis)

In [15]:
#sources = ag.loadSourcesFromFile("./sources_hyp.txt", rangeDist = (0, 25))

### Selecting the sources
The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

In [16]:
selectedSources = ag.selectSources('flux > 0')

2021-12-23 10:56:50,526 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.80893e-07]
2021-12-23 10:56:50,527 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.54772e-07]
2021-12-23 10:56:50,528 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.64949e-07]
2021-12-23 10:56:50,529 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [3.14104e-07]
2021-12-23 10:56:50,529 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [2.64853e-07]
2021-12-23 10:56:50,530 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.91737e-07]
2021-12-23 10:56:50,530 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [2.72475e-07]
2021-12-23 10:56:50,530 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [9.69539e-06]
2021-12-23 10:56:50,531 [DEBUG   ] [SourcesLibra

In [17]:
len(selectedSources)

9

In [18]:
selectedSources = ag.selectSources('name == "2AGLJ0835-4514"')

2021-12-23 10:56:50,660 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:50,662 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:50,664 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:50,665 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:50,668 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:50,669 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:50,670 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:50,672 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [19]:
len(selectedSources)
for s in selectedSources:
    print(s)


-----------------------------------------------------------
 Source name: 2AGLJ0835-4514 (PointSource)
  * Spectrum type: PLSuperExpCutoff
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 9.6954e-06
	- index1 : 1.71345
	- cutoffEnergy : 3913.06
	- index2 : 1.34774
	- Source position (l,b): (263.585, -2.84083)
	- Distance from map center (deg): 0.0702


In [20]:
selectedSources = ag.selectSources("flux > 0 AND dist <= 2")

2021-12-23 10:56:50,794 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.80893e-07, 24.215887883842015]
2021-12-23 10:56:50,795 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.54772e-07, 24.077750235539487]
2021-12-23 10:56:50,796 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.64949e-07, 23.460729370956138]
2021-12-23 10:56:50,796 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [3.14104e-07, 24.418297040070474]
2021-12-23 10:56:50,797 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.64853e-07, 21.76039561795526]
2021-12-23 10:56:50,797 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.91737e-07, 20.398740000053394]
2021-12-23 10:56:50,798 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.724

In [21]:
len(selectedSources)
for s in selectedSources:
    print(s)


-----------------------------------------------------------
 Source name: 2AGLJ0835-4514 (PointSource)
  * Spectrum type: PLSuperExpCutoff
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 9.6954e-06
	- index1 : 1.71345
	- cutoffEnergy : 3913.06
	- index2 : 1.34774
	- Source position (l,b): (263.585, -2.84083)
	- Distance from map center (deg): 0.0702


### Free/Fix source parameters
You can fix or free the following parametes:

* flux
* index
* index1
* index2
* cutoffEnergy
* pivotEnergy
* curvature
* index2
* pos for the position of the source

In [22]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", True, show=True)

2021-12-23 10:56:50,929 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:50,930 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:50,931 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:50,932 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:50,932 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:50,932 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:50,933 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:50,934 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [23]:
for s in _sources:
    print(s)


-----------------------------------------------------------
 Source name: 2AGLJ0835-4514 (PointSource)
  * Spectrum type: PLSuperExpCutoff
  * Free parameters: flux
  * Initial source parameters:
	- flux (ph/cm2s): 9.6954e-06
	- index1 : 1.71345
	- cutoffEnergy : 3913.06
	- index2 : 1.34774
	- Source position (l,b): (263.585, -2.84083)
	- Distance from map center (deg): 0.0702


In [24]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "index1", True)

2021-12-23 10:56:51,067 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:51,069 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:51,071 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:51,072 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:51,073 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:51,074 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:51,074 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:51,076 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [25]:
for s in _sources:
    print(s)


-----------------------------------------------------------
 Source name: 2AGLJ0835-4514 (PointSource)
  * Spectrum type: PLSuperExpCutoff
  * Free parameters: flux index1
  * Initial source parameters:
	- flux (ph/cm2s): 9.6954e-06
	- index1 : 1.71345
	- cutoffEnergy : 3913.06
	- index2 : 1.34774
	- Source position (l,b): (263.585, -2.84083)
	- Distance from map center (deg): 0.0702


In [26]:
_sources = ag.freeSources("flux > 0 AND dist <= 2", "flux", True)

2021-12-23 10:56:51,191 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.80893e-07, 24.215887883842015]
2021-12-23 10:56:51,192 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.54772e-07, 24.077750235539487]
2021-12-23 10:56:51,193 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.64949e-07, 23.460729370956138]
2021-12-23 10:56:51,194 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [3.14104e-07, 24.418297040070474]
2021-12-23 10:56:51,194 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.64853e-07, 21.76039561795526]
2021-12-23 10:56:51,195 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.91737e-07, 20.398740000053394]
2021-12-23 10:56:51,196 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.724

In [27]:
#len is 0 because the flux is already free
len(_sources)

0

In [28]:
for s in _sources:
    print(s)

Resetting the changes:

In [29]:
_sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", False)

2021-12-23 10:56:51,381 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:51,382 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:51,383 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:51,384 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:51,385 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:51,386 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:51,387 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:51,387 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [30]:
ag.freeSources('name == "2AGLJ0835-4514"', "index1", False)

2021-12-23 10:56:51,461 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:51,462 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:51,462 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:51,463 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:51,463 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:51,463 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:51,464 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:51,464 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [31]:
ag.freeSources("flux > 0 AND dist <= 2", "flux", False)

2021-12-23 10:56:51,543 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.80893e-07, 24.215887883842015]
2021-12-23 10:56:51,544 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.54772e-07, 24.077750235539487]
2021-12-23 10:56:51,545 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.64949e-07, 23.460729370956138]
2021-12-23 10:56:51,545 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [3.14104e-07, 24.418297040070474]
2021-12-23 10:56:51,546 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.64853e-07, 21.76039561795526]
2021-12-23 10:56:51,546 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [1.91737e-07, 20.398740000053394]
2021-12-23 10:56:51,547 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux', 'dist'] selectionParamsValues: [2.724

[]

### Deleting sources

In [32]:
deleted = ag.deleteSources('name == "2AGLJ1048-5836"').pop()
print(deleted)

2021-12-23 10:56:51,601 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:51,603 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1048-5836']
2021-12-23 10:56:51,604 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:51,604 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:51,605 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:51,605 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:51,606 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:51,607 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-

In [33]:
len(ag.getSources())

8

### Adding a source

Passing a dictionary:

In [34]:
newSourceDict = {
    "glon" : 273.0,
    "glat":  -1.1,
    "spectrumType" : "PowerLaw",
    "flux": 10e-08,
    "index": 2.1
}

if ag.addSource("NEWSOURCE", newSourceDict):
    print("Source loaded")

2021-12-23 10:56:51,753 [DEBUG   ] [SourcesLibrary] Loading source from a dictionary..
Source loaded


In [35]:
len(ag.getSources())

9

Passing a Source object:

In [36]:
type(deleted)

agilepy.core.source.Source.PointSource

In [37]:
if ag.addSource("NEWSOURCE", deleted):
    print("Source loaded")
else:
    print("Source is already present in the SourcesLibrary")

2021-12-23 10:56:51,933 [WARNING ] [SourcesLibrary] The source NEWSOURCE already exists. The 'sourceObject' will not be added to the SourcesLibrary.
Source is already present in the SourcesLibrary


In [38]:
deletedSources = ag.deleteSources('name == "NEWSOURCE"')

2021-12-23 10:56:51,993 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:51,994 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:51,995 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:51,995 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:51,996 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:51,996 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:51,997 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-23 10:56:51,997 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5906']
2021-12-

In [39]:
len(ag.getSources())

8

## Update the value of a spectrum parameter of a source

In [40]:
sources = ag.selectSources('name == "2AGLJ0835-4514"')
source = sources.pop()

2021-12-23 10:56:52,128 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:52,129 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:52,131 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:52,132 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:52,134 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:52,135 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:52,136 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-23 10:56:52,137 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5906']


In [41]:
print(source.get("index2"))

{'name': 'index2', 'value': 1.34774, 'err': None, 'free': 0, 'min': 0.0, 'max': 100.0, 'scale': -1, 'datatype': 'float', 'um': ''}


In [42]:
source.set("index2", {"value":1.34774, "min":10})
print(source)


-----------------------------------------------------------
 Source name: 2AGLJ0835-4514 (PointSource)
  * Spectrum type: PLSuperExpCutoff
  * Free parameters: none
  * Initial source parameters:
	- flux (ph/cm2s): 9.6954e-06
	- index1 : 1.71345
	- cutoffEnergy : 3913.06
	- index2 : 1.34774
	- Source position (l,b): (263.585, -2.84083)
	- Distance from map center (deg): 0.0702


## Fix the hypothesis for the analysis

This method returns the selected sources affected by the change

In [43]:
sources = ag.freeSources('name == "2AGLJ0835-4514"', "flux", True, show=True)

2021-12-23 10:56:52,343 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5954']
2021-12-23 10:56:52,345 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1045-5735']
2021-12-23 10:56:52,348 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1059-5233']
2021-12-23 10:56:52,350 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1029-5834']
2021-12-23 10:56:52,352 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1015-5852']
2021-12-23 10:56:52,354 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5752']
2021-12-23 10:56:52,356 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ0835-4514']
2021-12-23 10:56:52,359 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['name'] selectionParamsValues: ['2AGLJ1020-5906']

-------

Final source hypothesis for the analysis

In [44]:
selectedSources = ag.selectSources('flux > 0')
print(len(selectedSources))
for s in selectedSources:
    print(s)

2021-12-23 10:56:52,434 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.80893e-07]
2021-12-23 10:56:52,439 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.64949e-07]
2021-12-23 10:56:52,441 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [3.14104e-07]
2021-12-23 10:56:52,445 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [2.64853e-07]
2021-12-23 10:56:52,447 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.91737e-07]
2021-12-23 10:56:52,452 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [2.72475e-07]
2021-12-23 10:56:52,456 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [9.69539e-06]
2021-12-23 10:56:52,458 [DEBUG   ] [SourcesLibrary] userSelectionParams: ['flux'] selectionParamsValues: [1.38792e-07]
8

---------------------------------------------

## API: generate maps and perform maximum likelihood estimator analysis

### Generating maps

In [45]:
ag.printOptions("maps")

{ 'binsize': 0.4,
  'energybins': [[100, 300], [300, 1000]],
  'expstep': 4.0,
  'fovbinnumber': 1,
  'mapsize': 50,
  'offaxisangle': 30,
  'proj': 'ARC',
  'projtype': 'WCS',
  'spectralindex': 2.1,
  'timestep': 160,
  'useEDPmatrixforEXP': False}


In [46]:
maplistfile = ag.generateMaps()

Generating maps..please wait.


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

### Displaying maps

#### Single figure - multiple subplots

- normType selects a normalization function for the data, possible valuses are {‘linear’, ‘sqrt’, ‘power’, log’, ‘asinh’}, default ‘linear’.
- regFiles (list, optional) – A list containing region file(s) absolute path.
- regFileColors (list, optional) – A list with a color for each region file, the length must be the same of regFiles.
- catalogRegions (str, optional) – a catalog name. The regions that belongs to the catalog will be loaded. It defaults to None.
- catalogRegionsColor (str, optional) – the color of the regions loaded from the catalog.

In [ ]:
ag.displayCtsSkyMaps(smooth=2, regFiles=["$AGILE/catalogs/3EG_1.reg"], regFileColors=["red"], catalogRegions="2AGL", catalogRegionsColor="blue", normType="sqrt")

#### Multiple figures - one subplot per figure

In [ ]:
ag.displayExpSkyMaps(singleMode = False)

In [ ]:
ag.displayGasSkyMaps()

### Saving maps

In [ ]:
ag.displayCtsSkyMaps(smooth=3, saveImage=True)

### Estimation of background coefficients
You can use the [calcBkg()](https://agilepy.readthedocs.io/en/latest/api/analysis_api.html#api.AGAnalysis.AGAnalysis.calcBkg) method to estimate the background coefficients.

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

In [ ]:
isoBkg, galBkg, maplistfile = ag.calcBkg("2AGLJ0835-4514", galcoeff = [0.7, 0.7], pastTimeWindow = 0)

In [ ]:
# calcBkg with pastTimeWindow = 14
# ag.setOptions(galcoeff=[0.469176, 0.699403, 0.199108, 0.533379], isocoeff=[4.27314, 0.976644, 8.12386, 1.24627])

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

### Maximum likelyhood analysis

In [ ]:
sourceFiles = ag.mle()

In [ ]:
#print results
sources = ag.selectSources("sqrtTS > 0", show=True)
print(len(sources))
for s in sources:
    print(s)

Free also position parameter

In [ ]:
ag.freeSources('name == "2AGLJ0835-4514"', "pos", True, show=True)

In [ ]:
ag.mle()

In [ ]:
#print results
sources = ag.selectSources("sqrtTS > 0")
print(len(sources))
for s in sources:
    print(s)

Save the results of the analysis in a reg file.

In [ ]:
regFile = ag.writeSourcesOnFile("source_regions", "reg")

In [ ]:
print(regFile)

Display the result of the analysis saved in a reg file.

In [ ]:
ag.displayCtsSkyMaps(smooth=2, regFiles=[regFile], regFileColors=["red"], catalogRegions="2AGL", catalogRegionsColor="blue")

## Light curve

Fix the position and keep only flux free

In [ ]:
ag.freeSources('name == "2AGLJ0835-4514"', "pos", False, show=True)

In [ ]:
lightCurveData = ag.lightCurveMLE("2AGLJ0835-4514", binsize=86400)

In [ ]:
! cat $lightCurveData

In [ ]:
ag.displayLightCurve("mle")

time_start_mjd time_end_mjd sqrt(ts) flux flux_err flux_ul gal gal_error iso iso_error l_peak b_peak dist_peak l b r ell_dist a b phi exposure ExpRatio counts counts_err Index Index_Err Par2 Par2_Err Par3 Par3_Err Erglog Erglog_Err Erglog_UL time_start_utc time_end_utc time_start_tt time_end_tt Fix index ULConfidenceLevel SrcLocConfLevel start_l start_b start_flux typefun par2 par3 galmode2 galmode2fit isomode2 isomode2fit edpcor fluxcor integratortype expratioEval expratio_minthr expratio_maxthr expratio_size Emin emax fovmin fovmax albedo binsize expstep phasecode

In [ ]:
ag.displayGenericColumns(lightCurveData, ["exposure"], um=["cm^2 s sr"])

## Aperture Photometry

In [ ]:
#ap_file, _ = ag.aperturePhotometry()

In [ ]:
#! cat $ap_file

In [ ]:
#ag.displayLightCurve("ap")

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()